## Overview

In [14]:
import json, warnings, os
from pathlib import Path

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from stables.utils.postgres import get_sqlalchemy_engine
from stables.config import PostgresConfig
db_config = PostgresConfig()
engine = get_sqlalchemy_engine(db_config)

In [ ]:
query = """
SELECT * FROM ethena_raw.circulating
"""
df_ethena= pd.read_sql(query, engine)
df_ethena['date'] = pd.to_datetime(df_ethena['date'], unit='s')

### 1. TVL

In [ ]:
df = df_ethena.copy()
l = df['date'].max()

tvl_usde = df[
    (df['date'] == l) & (df['id'] == '146')
]['circulating'].sum()

tvl_usdtb = df[
    (df['date'] == l) & (df['id'] == '221')
]['circulating'].sum()

tvl = tvl_usde + tvl_usdtb


n_chains_usde = df[
    (df['date'] == l) & (df['id'] == '146')
]['chain'].nunique()
n_chains_usdtb = df[
    (df['date'] == l) & (df['id'] == '221')
]['chain'].nunique()

In [ ]:


df = df_ethena.groupby(['date', 'symbol']).agg(
    tvl=('circulating', 'sum')
).reset_index()
fig = px.bar(df, x="date", y="tvl", color="symbol", title=f"Current TVL: {tvl/1e9:.2f}B, USDe: {tvl_usde/1e9:.2f}B, USDTB: {tvl_usdtb/1e9:.2f}B",labels="")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Stablecoin"
)
fig.show()

### 2. TVL by chains

In [ ]:
df = df_ethena.copy()
df = df.groupby(['date', 'chain']).agg(
    tvl=('circulating', 'sum')
).reset_index()
fig = px.bar(df, x="date", y="tvl", color="chain", title="Ethena TVL by chains")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Chain"
)
fig.show()

### 3. USDe staking percentage

In [ ]:
df1 = df_ethena[df_ethena['id'] == '146'].groupby(['date']).agg(tvl=('circulating', 'sum')).reset_index()

query = """
SELECT * FROM ethena_raw.susde_pool
"""
df2 = pd.read_sql(query, engine)
df2['date'] = df2['timestamp'].dt.date
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
df = df1.merge(df2[['date', 'tvl_usd']], on='date', how='outer')#, suffixes=('_usde', '_sus)
df.rename(columns={'tvl_usd': 'susde', 'tvl': 'usde'}, inplace=True)
df['staking_perc'] = (df['susde'] / df['usde']) *100
current_staking_perc = df[df['date'] == df['date'].max()]['staking_perc'].values[0]

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=df2['date'], y=df2['tvl_usd'], mode='lines', name='sUSDe'))
fig.add_trace(go.Bar(x=df1['date'], y=df1['tvl'], name='USDe', opacity=0.5))

fig.add_trace(go.Scatter(
      x=df['date'],
      y=df['staking_perc'],
      mode='lines',
      name='Staking %',
      yaxis='y2'
  ))
fig.update_layout(
      yaxis2=dict(
          title="%",
          overlaying='y',
          side='right'
      ),
    title="Current USDe TVL: {:.2f}B, staking {:.2f}%".format(tvl_usde/1e9, current_staking_perc),
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    # legend_title_text="Stablecoin"
  )


### 4. Revenue

In [11]:
query = """
SELECT * FROM ethena_raw.protocol_revenue_ethena
"""
df = pd.read_sql(query, engine)
# df['date'] = df['timestamp'].dt.date
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date

df

,timestamp,revenue,protocol,_dlt_load_id,_dlt_id,chain,sub_protocol,date
0,1700870400,0,ethena,1753107066.1861103,hq9hVFxJP6szDQ,Ethereum,Ethena USDe,2023-11-25
1,1700956800,0,ethena,1753107066.1861103,6CqQJnPnRQPk8A,Ethereum,Ethena USDe,2023-11-26
2,1701043200,0,ethena,1753107066.1861103,Y8kah/Y+XdqnCQ,Ethereum,Ethena USDe,2023-11-27
3,1701129600,103,ethena,1753107066.1861103,1Sl2GrcPSfPbgw,Ethereum,Ethena USDe,2023-11-28
4,1701216000,3,ethena,1753107066.1861103,Oi0EnSGyfgghdw,Ethereum,Ethena USDe,2023-11-29
...,...,...,...,...,...,...,...,...
594,1752624000,1661,ethena,1753107066.1861103,gRonI5cPkRlIYQ,Ethereum,Ethena USDe,2025-07-16
595,1752710400,7601364,ethena,1753107066.1861103,ck5+U7QV4YFz0Q,Ethereum,Ethena USDe,2025-07-17
596,1752796800,145676,ethena,1753107066.1861103,JVS2kxYOIg1s/w,Ethereum,Ethena USDe,2025-07-18
597,1752883200,165763,ethena,1753107066.1861103,wXu7PtLRGLokng,Ethereum,Ethena USDe,2025-07-19


In [16]:
px.line(df, x="date", y="revenue", color="protocol")

### 4. ENA